# Surrogate Construction for Genz Functions

This notebook constructs a PC surrogate for Genz functions using full and sparse quadrature and gives the root mean square error between the surrogate and the actual function. Both the Genz function and PCE are defined on [-1,1].

In [13]:
from __future__ import print_function
import numpy as np
import math  
from scipy.stats import qmc

import PyUQTk.pce as uqtkpce
import PyUQTk.PyPCE.pce_tools as pce_tools
from PyUQTk.utils.func import *

## Inputs

In [14]:
nord = 3            # Order of the PCE
pc_type = "LU"      # Polynomial type
pc_alpha = 0.0      # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
pc_beta = 1.0 
# Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
param = nord+1           # Number of quadrature points per dimension for full quadrature or level for sparse quadrature
nSam = 10000       # Number of random samples
ndim = 3         # Number of dimensions
model= 'genz_osc'   # Choices are 'genz_osc', 'genz_exp', 'genz_cont','genz_gaus','genz_cpeak', 'genz_ppeak'

In [5]:
rng = qmc.LatinHypercube(d=ndim, seed=42)

## Quadrature and Galerkin Projection
#### Full Quadrature

First, we instantiate the PC object with full quadrature.

In [6]:
pc_model = uqtkpce.PCSet("NISPnoq", nord, ndim, pc_type, pc_alpha,pc_beta)
pc_model.SetQuadRule(pc_type, 'full', param)

Then, we get a NumPy array of quadrature points, qdpts. Totquat is the total number of quadrature points.

In [7]:
qdpts, totquat= pce_tools.UQTkGetQuadPoints(pc_model)

We evaluate the forward model, the chosen Genz function. The input parameters for the model are all 1, for simpicity, and must be in the form of an array of dimension ndim+1, where the first entry is the shift.
We obtain f_evals, an array of evaluations.

In [8]:
f_evals=func(qdpts,model,np.ones(ndim+1))

We obtain the PC coefficents with Galerkin projection or regression

In [9]:
c_k = pce_tools.UQTkGalerkinProjection(pc_model,f_evals)

We then generate germ samples in [-1, 1] at which to evaluate the PCE. These are the points at which we are checking the accuracy of the surrogate.

In [10]:
#germ_samples=np.random.normal(0,1, (nSam,ndim))
#rng = np.random.default_rng()
#germ_samples=2*rng.random((nSam, ndim))-1

germ_samples=2*rng.random(n=nSam)-1 #draw n samples from [-1,1]
pce_evals=pce_tools.UQTkEvaluatePCE(pc_model,c_k,germ_samples)

We find the actual values of the model at each germ sample and use that to calculate the RMS error between the surrogate and model.

In [11]:
f_actual=func(germ_samples,model,np.ones(ndim+1))
MSE = np.square(np.subtract(f_actual,pce_evals)).mean()
RMSE=math.sqrt(MSE)

print("The RMS error between a", ndim, "-dimensional", model, "function and a full PC surrogate of order", nord, "is")
RMSE

The RMS error between a 3 -dimensional genz_osc function and a full PC surrogate of order 3 is


0.000888449813273754

#### Sparse Quadrature
We repeat the process with sparse quadrature.

In [12]:
pc_model2 = uqtkpce.PCSet("NISPnoq", nord,ndim,pc_type, pc_alpha,pc_beta)
pc_model2.SetQuadRule(pc_type, 'sparse', param)
qdpts2, totquat2= pce_tools.UQTkGetQuadPoints(pc_model2)
f_evals2=func(qdpts2,model,np.ones(ndim+1))
c_k2 = pce_tools.UQTkGalerkinProjection(pc_model2,f_evals2)
#germ_samples2=np.random.normal(0,1, (nSam,ndim))
#germ_samples2=2*rng.random((nSam, ndim))-1
germ_samples2=2*rng.random(n=nSam)-1 #draw n samples from [-1,1]
pce_evals2=pce_tools.UQTkEvaluatePCE(pc_model2,c_k2,germ_samples2)
f_actual2=func(germ_samples2,model,np.ones(ndim+1))
MSE2 = np.square(np.subtract(f_actual2,pce_evals2)).mean()
RMSE2=math.sqrt(MSE2)
print("The RMS error between a", ndim, "-dimensional", model, "function and a sparse PC surrogate of order", nord, "is")
RMSE2

The RMS error between a 3 -dimensional genz_osc function and a sparse PC surrogate of order 3 is


0.000877791186336563

## Random Sampling
#### Regression

Similarly, we define a PC object for regression and then get a collection of random sample points in [-1,1].

In [15]:
# Instantiate
pc_model3 = uqtkpce.PCSet("NISPnoq", nord, ndim, pc_type, pc_alpha, pc_beta)

nTrain=pc_model3.GetNumberPCTerms()
#randpts=np.random.normal(loc=0, scale=0.5, size=(int(nTrain*1.1), ndim))
#np.random.seed(42)
#randpts= 2*rng.random((int(nTrain), ndim))-1

randpts=2*rng.random(n=int(nTrain*1.2))-1 #draw n samples from [-1,1]
f_evals3=func(randpts,model,np.ones(ndim+1))

In [16]:
randpts.shape

(24, 3)

In [18]:
nTrain

20

Then, we use regression to find the PC coefficients and evaluate the PC at nSam sample points. The error between these evaluations and the actual model is determined.

In [12]:
#Regression
c_k3 = pce_tools.UQTkRegression(pc_model3, f_evals3, randpts)

# Validation points
#germ_samples3=np.random.normal(0,1, (nSam,ndim))
#germ_samples3=2*rng.random((nSam, ndim))-1
germ_samples3=2*rng.random(n=nSam)-1
pce_evals3=pce_tools.UQTkEvaluatePCE(pc_model3,c_k3,germ_samples3)

#Error
f_actual3=func(germ_samples3,model,np.ones(ndim+1))
MSE3 = np.square(np.subtract(f_actual3,pce_evals3)).mean()
RMSE3=math.sqrt(MSE3)
print("The RMS error between a", ndim, "-dimensional", model, "function and a regression-based PC surrogate of order", nord, "is")
RMSE3

The RMS error between a 3 -dimensional genz_osc function and a regression-based PC surrogate of order 3 is


0.0016187507585282224

#### BCS

In [13]:
pc_model4 = uqtkpce.PCSet("NISPnoq", nord, ndim, pc_type, pc_alpha, pc_beta)
nTrain2=int(pc_model4.GetNumberPCTerms()*1.2)
randpts2=2*rng.random(n=nTrain2)-1
f_evals4=func(randpts2, model, np.ones(ndim+1))

In [14]:
sigma = np.array([1e-8])   # inital noise variance; updated in BCS
eta = 1e-8                 # threshold for stopping the algorithm
lambda_init = np.array([]) # set lambda to a fixed nonnegative value

scale = 0.1     # diagonal loading parameter
adaptive = 1    # generative basis for adaptive CS, 0 or 1
optimal = 1     # use the rigorous implementation of adaptive CS, 0 or 1

c_k4 = pce_tools.UQTkBCS(pc_model4, f_evals4, randpts2, sigma, eta, lambda_init, adaptive, optimal, scale)

germ_samples4=2*rng.random(n=nSam)-1
pce_evals4=pce_tools.UQTkEvaluatePCE(pc_model4,c_k4,germ_samples4)

#Error
f_actual4=func(germ_samples4,model,np.ones(ndim+1))
MSE4 = np.square(np.subtract(f_actual4,pce_evals4)).mean()
RMSE4=math.sqrt(MSE4)
print("The RMS error between a", ndim, "-dimensional", model, "function and a BCS-based PC surrogate of order", nord, "is")
RMSE4

Iteration # 0
Iteration # 10
Iteration # 20
The RMS error between a 3 -dimensional genz_osc function and a BCS-based PC surrogate of order 3 is


0.00297733833070045